In [29]:
for i in range(1,18):
    with open(f'../data/100_known_ts_{i}.pkl','rb') as f:
        data,none,dags,causal_dfs = pickle.load(f)
    new_causal_dfs = []
    for dag in dags:
        new_causal_df = get_causal_dfs(dag,5,3)
        new_causal_dfs.append(new_causal_df)
    with open(f'../data/100_known_ts_{i}.pkl','wb') as f:
        pickle.dump((data,none,dags,new_causal_dfs),f)

FileNotFoundError: [Errno 2] No such file or directory: '100_known_ts_1.pkl'

In [124]:
import random
import math
import networkx as nx

In [125]:
# Helper function to compute mean over given indices
def mean_over_indices(Y_t, indices):
    return sum(Y_t[i] for i in indices) / len(indices)

# Indicator function
def I(condition):
    return 1 if condition else 0

# Sign function
def sign(x):
    return -1 if x < 0 else (1 if x > 0 else 0)


$Y_{t+1}[j]=-0.4 \frac{\left(3-\bar{Y}_t\left[\mathcal{N}_j\right]^2\right)}{\left(1+\bar{Y}_t\left[\mathcal{N}_j\right]^2\right)}+0.6 \frac{3-\left(\bar{Y}_{t-1}\left[\mathcal{N}_j\right]-0.5\right)^3}{1+\left(\bar{Y}_{t-1}\left[\mathcal{N}_j\right]-0.5\right)^4}+W_{t+1}[j]$

In [126]:
# Time series update function for the first formula
def time_series_update_1(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)
    Y_bar_t_minus_1 = mean_over_indices(Y[t-1], N_j)

    term1 = -0.4 * (3 - Y_bar_t**2) / (1 + Y_bar_t**2)
    term2 = 0.6 * (3 - (Y_bar_t_minus_1 - 0.5)**3) / (1 + (Y_bar_t_minus_1 - 0.5)**4)

    return term1 + term2 + W[t][j]

In [127]:

def build_DAG_time_series_1(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx
    G = nx.DiGraph()

    # Initialize
    for t in range(T+1):
        for j in range(N):
            G.add_node(f"Y[{t}][{j}]")

    # Add nodes and edges to the DAG based on the dependencies
    for t in range(T):
        for j in range(N):
            # if t > 0:
                # Add edges from the current and previous Y to the next Y
                for nj in N_j:
                    G.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
                    if t != T-1:
                        G.add_edge(f"Y[{t}][{nj}]", f"Y[{t+2}][{j}]")
                
                
    return G

$Y_{t+1}[j]=\left(0.4-2 \cos \left(40 \bar{Y}_{t-5}\left[\mathcal{N}_j\right]\right) \exp \left(-30 \bar{Y}_{t-5}\left[\mathcal{N}_j\right]^2\right)\right) \bar{Y}_{t-5}\left[\mathcal{N}_j\right]+\left(0.5-0.5 \exp \left(-50 \bar{Y}_{t-9}\left[\mathcal{N}_j\right]^2\right)\right) \bar{Y}_{t-9}\left[\mathcal{N}_j\right]+W_{t+1}[j]$

In [128]:


# Time series update function for the second formula
def time_series_update_2(Y, t, j, N_j, W):
    Y_bar_t_minus_5 = mean_over_indices(Y[t-5], N_j)
    Y_bar_t_minus_9 = mean_over_indices(Y[t-9], N_j)

    term1 = (0.4 - 2 * math.exp(-50 * Y_bar_t_minus_5**2)) * Y_bar_t_minus_5
    term2 = (0.5 - 0.5 * math.exp(-50 * Y_bar_t_minus_9**2)) * Y_bar_t_minus_9

    return term1 + term2 + W[t][j]


In [129]:

def build_DAG_time_series_2(T, N_j, N):
    # Create a new Directed Acyclic Graph (DAG) using networkx for the second formula
    G2 = nx.DiGraph()

    # Initialize
    for t in range(T+1):
        for j in range(N):
            G2.add_node(f"Y[{t}][{j}]")

    # Add nodes and edges to the DAG based on the dependencies for the second formula
    for t in range(1, T):
        for j in range(N):
            
            # For t >= 5, add edges from Y[t-5] to Y[t]
            if t >= 5:
                for nj in N_j:
                    G2.add_node(f"Y[{t-5}][{nj}]")
                    G2.add_edge(f"Y[{t-5}][{nj}]", f"Y[{t}][{j}]")
            
            # For t >= 9, add edges from Y[t-9] to Y[t]
            if t >= 9:
                for nj in N_j:
                    G2.add_node(f"Y[{t-9}][{nj}]")
                    G2.add_edge(f"Y[{t-9}][{nj}]", f"Y[{t}][{j}]")
            
            

    return G2

$Y_{t+1}[j]=1.5 \sin \left(\pi / 2 \bar{Y}_{t-1}\left[\mathcal{N}_j\right]\right)-\sin \left(\pi / 2 \bar{Y}_{t-2}\left[\mathcal{N}_j\right]\right)+W_{t+1}[j]$

In [130]:
# Time series update function for the third formula
def time_series_update_3(Y, t, j, N_j, W):
    Y_bar_t_minus_1 = mean_over_indices(Y[t-1], N_j)
    Y_bar_t_minus_2 = mean_over_indices(Y[t-2], N_j)

    term1 = 1.5 * math.sin(math.pi / 2 * Y_bar_t_minus_1)
    term2 = -math.sin(math.pi / 2 * Y_bar_t_minus_2)

    return term1 + term2 + W[t][j]


In [131]:
def build_DAG_time_series_3(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the third formula
    G3 = nx.DiGraph()
    
    # Initialize
    for t in range(T+1):
        for j in range(N):
            G3.add_node(f"Y[{t}][{j}]")
    
    # Add nodes and edges to the DAG based on the dependencies for the third formula
    for t in range(T):  # Start from t=2 since the formula depends on t-1 and t-2
        for j in range(N):

            # Add edges from the previous and the one before last Y to the next Y
            for nj in N_j:
                if t >= 1:
                    G3.add_edge(f"Y[{t-1}][{nj}]", f"Y[{t+1}][{j}]")
                if t >= 2:
                    G3.add_edge(f"Y[{t-2}][{nj}]", f"Y[{t+1}][{j}]")
            

    return G3


$Y_{t+1}[j]=2 \exp \left(-0.1 \bar{Y}_t\left[\mathcal{N}_j\right]^2\right) \bar{Y}_t\left[\mathcal{N}_j\right]-\exp \left(-0.1 \bar{Y}_{t-1}\left[\mathcal{N}_j\right]^2\right) \bar{Y}_{t-1}\left[\mathcal{N}_j\right]+W_{t+1}[j]$

In [132]:
# Time series update function for the fourth formula you provided
def time_series_update_4(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)
    Y_bar_t_minus_1 = mean_over_indices(Y[t-1], N_j)

    term1 = 2 * math.exp(-0.1 * Y_bar_t**2) * Y_bar_t
    term2 = -math.exp(-0.1 * Y_bar_t_minus_1**2) * Y_bar_t_minus_1

    return term1 + term2 + W[t][j]



In [133]:
def build_DAG_time_series_4(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the fourth formula
    G4 = nx.DiGraph()
    
    # Initialize
    for t in range(T+1):
        for j in range(N):
            G4.add_node(f"Y[{t}][{j}]")
    
    # Add nodes and edges to the DAG based on the dependencies for the fourth formula
    for t in range(T):  # Start from t=1 since the formula depends on t and t-1
        for j in range(N):
            # Add edges from the current and previous Y to the next Y
            for nj in N_j:
                G4.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
                if t >= 1:
                    G4.add_edge(f"Y[{t-1}][{nj}]", f"Y[{t+1}][{j}]")
            

    return G4

$Y_{t+1}[j]=-2 \bar{Y}_t\left[\mathcal{N}_j\right] I\left(\bar{Y}_t\left[\mathcal{N}_j\right]<0\right)+0.4 \bar{Y}_t\left[\mathcal{N}_j\right] I\left(\bar{Y}_t\left[\mathcal{N}_j\right]<0\right)+W_{t+1}[j]$

In [134]:
# Time series update function for the fifth formula you provided
def time_series_update_5(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)

    term1 = -2 * Y_bar_t * I(Y_bar_t < 0)
    term2 = 0.4 * Y_bar_t * I(Y_bar_t < 0)

    return term1 + term2 + W[t][j]


In [135]:
def build_DAG_time_series_5(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the fifth formula
    G5 = nx.DiGraph()
    
    # Initialize
    for t in range(T+1):
        for j in range(N):
            G5.add_node(f"Y[{t}][{j}]")
    
    # Add nodes and edges to the DAG based on the dependencies for the fifth formula
    for t in range(T):  
        for j in range(N):
            # Add edges from the current Y to the next Y
            for nj in N_j:
                G5.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
            

    return G5


$Y_{t+1}[j]=0.8 \log \left(1+3 \bar{Y}_t\left[\mathcal{N}_j\right]^2\right)-0.6 \log \left(1+3 \bar{Y}_{t-2}\left[\mathcal{N}_j\right]^2\right)+W_{t+1}[j]$

In [136]:
# Time series update function for the sixth formula you provided
def time_series_update_6(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)
    Y_bar_t_minus_2 = mean_over_indices(Y[t-2], N_j)

    term1 = 0.8 * math.log(1 + 3 * Y_bar_t**2)
    term2 = -0.6 * math.log(1 + 3 * Y_bar_t_minus_2**2)

    return term1 + term2 + W[t][j]


In [137]:
def build_DAG_time_series_6(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the sixth formula
    G6 = nx.DiGraph()

    # Initialize
    for t in range(T+1):
        for j in range(N):
            G6.add_node(f"Y[{t}][{j}]")
    
    # Add nodes and edges to the DAG based on the dependencies for the sixth formula
    for t in range(T):
        for j in range(N):
            for nj in N_j:
                G6.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
                if t >= 2:
                    G6.add_edge(f"Y[{t-2}][{nj}]", f"Y[{t+1}][{j}]")

    return G6

$Y_{t+1}[j]=\left(0.4-2 \cos \left(40 \bar{Y}_{t-5}\left[\mathcal{N}_j\right]\right) \exp \left(-30 \bar{Y}_{t-5}\left[\mathcal{N}_j\right]^2\right)\right) \bar{Y}_{t-5}\left[\mathcal{N}_j\right]+\left(0.5-0.5 \exp \left(-50 \bar{Y}_{t-9}\left[\mathcal{N}_j\right]^2\right)\right) \bar{Y}_{t-9}\left[\mathcal{N}_j\right]+W_{t+1}[j]$

In [138]:
def time_series_update_7(Y, t, j, N_j, W):
    Y_bar_t_minus_5 = mean_over_indices(Y[t-5], N_j)
    Y_bar_t_minus_9 = mean_over_indices(Y[t-9], N_j)

    term1_part1 = 0.4 - 2 * math.cos(40 * Y_bar_t_minus_5) * math.exp(-30 * Y_bar_t_minus_5**2)
    term1 = term1_part1 * Y_bar_t_minus_5
    term2 = (0.5 - 0.5 * math.exp(-50 * Y_bar_t_minus_9**2)) * Y_bar_t_minus_9

    return term1 + term2 + W[t][j]


In [139]:
def build_DAG_time_series_7(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the sixth formula
    G7 = nx.DiGraph()
    
    # Initialize
    for t in range(T+1):
        for j in range(N):
            G7.add_node(f"Y[{t}][{j}]")
    
    # Add nodes and edges to the DAG based on the dependencies for the seventh formula
    for t in range(T):
        for j in range(N):
            for nj in N_j:
                if t >= 5:
                    G7.add_edge(f"Y[{t-5}][{nj}]", f"Y[{t+1}][{j}]")
                if t >= 9:
                    G7.add_edge(f"Y[{t-9}][{nj}]", f"Y[{t+1}][{j}]")

    return G7


$Y_{t+1}[j]=\left(0.5-1.1 \exp \left(-50 \bar{Y}_t\left[\mathcal{N}_j\right]^2\right)\right) \bar{Y}_t\left[\mathcal{N}_j\right]+\left(0.3-0.5 \exp \left(-50 \bar{Y}_{t-2}\left[\mathcal{N}_j\right]^2\right)\right) \bar{Y}_{t-2}\left[\mathcal{N}_j\right]+W_{t+1}[j]$

In [140]:
def time_series_update_8(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)
    Y_bar_t_minus_2 = mean_over_indices(Y[t-2], N_j)

    term1 = (0.5 - 1.1 * math.exp(-50 * Y_bar_t**2)) * Y_bar_t
    term2 = (0.3 - 0.5 * math.exp(-50 * Y_bar_t_minus_2**2)) * Y_bar_t_minus_2

    return term1 + term2 + W[t][j]


In [141]:
def build_DAG_time_series_8(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the sixth formula
    G8 = nx.DiGraph()
    
    # Initialize
    for t in range(T+1):
        for j in range(N):
            G8.add_node(f"Y[{t}][{j}]")
    
    # Add nodes and edges for the eighth formula
    for t in range(T):  # Start from t=2 since the formula depends on t and t-2
        for j in range(N):
            for nj in N_j:
                G8.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
                if t >= 2:
                    G8.add_edge(f"Y[{t-2}][{nj}]", f"Y[{t+1}][{j}]")

    return G8


$Y_{t+1}[j]=0.3 \bar{Y}_t\left[\mathcal{N}_j\right]+0.6 \bar{Y}_{t-1}\left[\mathcal{N}_j\right]+\frac{\left(0.1-0.9 \bar{Y}_t\left[\mathcal{N}_j\right]+0.8 \bar{Y}_{t-1}\left[\mathcal{N}_j\right]\right)}{\left(1+\exp \left(-10 \bar{Y}_t\left[\mathcal{N}_j\right]\right)\right)}+W_{t+1}[j]$

In [142]:

# Time series update function for the ninth formula you provided
def time_series_update_9(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)
    Y_bar_t_minus_1 = mean_over_indices(Y[t-1], N_j)

    term1 = 0.3 * Y_bar_t
    term2 = 0.6 * Y_bar_t_minus_1
    term3_numerator = 0.1 - 0.9 * Y_bar_t + 0.8 * Y_bar_t_minus_1
    term3_denominator = 1 + math.exp(-10 * Y_bar_t)
    term3 = term3_numerator / term3_denominator

    return term1 + term2 + term3 + W[t][j]


In [143]:
def build_DAG_time_series_9(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the ninth formula
    G9 = nx.DiGraph()

    for t in range(T+1):
        for j in range(N):
            G9.add_node(f"Y[{t}][{j}]")
    
    # Add nodes and edges to the DAG based on the dependencies for the ninth formula
    for t in range(T):  # Start from t=1 since the formula depends on t and t-1
        for j in range(N):
            # Add edges from the current and previous Y to the next Y
            for nj in N_j:
                G9.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
                if t >= 1:
                    G9.add_edge(f"Y[{t-1}][{nj}]", f"Y[{t+1}][{j}]")
            

    return G9


$Y_{t+1}[j]=\operatorname{sign}\left(\bar{Y}_t\left[\mathcal{N}_j\right]\right)+W_{t+1}[j]$

In [144]:
# Time series update function for the tenth formula you provided
def time_series_update_10(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)
    return sign(Y_bar_t) + W[t][j]


In [145]:
def build_DAG_time_series_10(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the tenth formula
    G10 = nx.DiGraph()
    
    for t in range(T+1):
        for j in range(N):
            G10.add_node(f"Y[{t}][{j}]")
    
    # Add nodes and edges to the DAG based on the dependencies for the tenth formula
    for t in range(T):  # Start from t=1 since the formula depends on t
        for j in range(N):
            # Add edges from the current Y to the next Y
            for nj in N_j:
                G10.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
            

    return G10
                

$Y_{t+1}[j]=0.8 \bar{Y}_t\left[\mathcal{N}_j\right]-\frac{0.8 \bar{Y}_t\left[\mathcal{N}_j\right]}{\left(1+\exp \left(-10 \bar{Y}_t\left[\mathcal{N}_j\right]\right)\right)}+W_{t+1}[j]$

In [146]:
# Time series update function for the eleventh formula you provided
def time_series_update_11(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)
    term1 = 0.8 * Y_bar_t
    term2_denominator = 1 + math.exp(-10 * Y_bar_t)
    term2 = -0.8 * Y_bar_t / term2_denominator

    return term1 + term2 + W[t][j]


In [147]:
def build_DAG_time_series_11(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the eleventh formula
    G11 = nx.DiGraph()
    
    for t in range(T+1):
        for j in range(N):
            G11.add_node(f"Y[{t}][{j}]")
    
    # Add nodes and edges to the DAG based on the dependencies for the eleventh formula
    for t in range(T):  # Start from t=1 since the formula depends on t
        for j in range(N):

            # Add edges from the current Y to the next Y
            for nj in N_j:
                G11.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
            

    return G11

$Y_{t+1}[j]=0.3 \bar{Y}_t\left[\mathcal{N}_j\right]+0.6 \bar{Y}_{t-1}\left[\mathcal{N}_j\right]+\frac{\left(0.1-0.9 \bar{Y}_t\left[\mathcal{N}_j\right]+0.8 \bar{Y}_{t-1}\left[\mathcal{N}_j\right]\right)}{\left(1+\exp \left(-10 \bar{Y}_t\left[\mathcal{N}_j\right]\right)\right)}+W_{t+1}[j]$

In [148]:
# Time series update function for the twelfth formula you provided
def time_series_update_12(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)
    Y_bar_t_minus_1 = mean_over_indices(Y[t-1], N_j)

    term1 = 0.3 * Y_bar_t
    term2 = 0.6 * Y_bar_t_minus_1
    term3_numerator = 0.1 - 0.9 * Y_bar_t + 0.8 * Y_bar_t_minus_1
    term3_denominator = 1 + math.exp(-10 * Y_bar_t)
    term3 = term3_numerator / term3_denominator

    return term1 + term2 + term3 + W[t][j]


In [149]:
def build_DAG_time_series_12(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the twelfth formula
    G12 = nx.DiGraph()
    
    for t in range(T+1):
        for j in range(N):
            G12.add_node(f"Y[{t}][{j}]")
    
    # Add nodes and edges to the DAG based on the dependencies for the twelfth formula
    for t in range(T):  # Start from t=1 since the formula depends on t and t-1
        for j in range(N):

            # Add edges from the current and previous Y to the next Y
            for nj in N_j:
                G12.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
                if t >= 1:
                    G12.add_edge(f"Y[{t-1}][{nj}]", f"Y[{t+1}][{j}]")
            

    return G12

$Y_{t+1}[j]=0.38 \bar{Y}_t\left[\mathcal{N}_j\right]\left(1-\bar{Y}_{t-1}\left[\mathcal{N}_j\right]\right)+W_{t+1}[j]$

In [150]:
def time_series_update_13(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)
    Y_bar_t_minus_1 = mean_over_indices(Y[t-1], N_j)

    term1 = 0.38 * Y_bar_t * (1 - Y_bar_t_minus_1)

    return term1 + W[t][j]


In [151]:
def build_DAG_time_series_13(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the thirteenth formula
    G13 = nx.DiGraph()
    
    for t in range(T+1):
        for j in range(N):
            G13.add_node(f"Y[{t}][{j}]")
    
    # Add nodes and edges to the DAG based on the dependencies for the thirteenth formula
    for t in range(T):  # Start from t=1 since the formula depends on t and t-1
        for j in range(N):
            # Add the current and previous Y nodes if not already present

            # Add edges from the current and previous Y to the next Y
            for nj in N_j:
                G13.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
                if t >= 1:
                    G13.add_edge(f"Y[{t-1}][{nj}]", f"Y[{t+1}][{j}]")
            
    return G13

$Y_{t+1}[j]=\left\{\begin{array}{l}-0.5 \bar{Y}_t\left[\mathcal{N}_j\right] \quad \text { if } \quad \bar{Y}_t\left[\mathcal{N}_j\right]<1 \\ 0.4 \bar{Y}_t\left[\mathcal{N}_j\right]\end{array}\right.$

In [152]:
def time_series_update_14(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)

    if Y_bar_t < 1:
        return -0.5 * Y_bar_t + W[t][j]
    else:
        return 0.4 * Y_bar_t + W[t][j]


In [153]:
def build_DAG_time_series_14(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the thirteenth formula
    G14 = nx.DiGraph()
    
    for t in range(T+1):
        for j in range(N):
            G14.add_node(f"Y[{t}][{j}]")
    

    # Add nodes and edges to the DAG based on the dependencies for the thirteenth formula
    for t in range(T):  # Start from t=1 since the formula depends on t and t-1
        for j in range(N):
            # Add the current and previous Y nodes if not already present

            # Add edges from the current and previous Y to the next Y
            for nj in N_j:
                G14.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
            
    return G14

$Y_{t+1}[j]=\left\{\begin{array}{l}0.9 \bar{Y}_t\left[\mathcal{N}_j\right]+W_{t+1}[j] \text { if } \quad\left|\bar{Y}_t\left[\mathcal{N}_j\right]\right|<1 \\ -0.3 \bar{Y}_t\left[\mathcal{N}_j\right]+W_{t+1}[j]\end{array}\right.$

In [154]:
def time_series_update_15(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)

    if abs(Y_bar_t) < 1:
        return 0.9 * Y_bar_t + W[t][j]
    else:
        return -0.3 * Y_bar_t + W[t][j]


In [155]:
def build_DAG_time_series_15(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the thirteenth formula
    G15 = nx.DiGraph()
    
    for t in range(T+1):
        for j in range(N):
            G15.add_node(f"Y[{t}][{j}]")
    
    
    # Add nodes and edges to the DAG based on the dependencies for the thirteenth formula
    for t in range(T):  # Start from t=1 since the formula depends on t and t-1
        for j in range(N):
            # Add the current and previous Y nodes if not already present

            # Add edges from the current and previous Y to the next Y
            for nj in N_j:
                G15.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
            
    return G15

$\begin{aligned} Y_{t+1}[j] & =\left\{\begin{array}{l}-0.5 \bar{Y}_t\left[\mathcal{N}_j\right]+W_{t+1}[j] \quad \text { if } \quad x_t=1 \\ 0.4 \bar{Y}_t\left[\mathcal{N}_j\right]+W_{t+1}[j]\end{array}\right. \\ x_{t+1} & =1-x_t, x_0=1\end{aligned}$

In [156]:
def update_x(x_t):
    return 1 - x_t

# Time series update function for the sixteenth formula you provided
def time_series_update_16(Y, t, j, N_j, W, x_t):
    Y_bar_t = mean_over_indices(Y[t], N_j)

    if x_t == 1:
        return -0.5 * Y_bar_t + W[t][j]
    else:
        return 0.4 * Y_bar_t + W[t][j]


In [157]:
def build_DAG_time_series_16(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the thirteenth formula
    G16 = nx.DiGraph()
    
    for t in range(T+1):
        for j in range(N):
            G16.add_node(f"Y[{t}][{j}]")
    
    
    # Add nodes and edges to the DAG based on the dependencies for the thirteenth formula
    for t in range(T):  # Start from t=1 since the formula depends on t and t-1
        for j in range(N):

            # Add edges from the current and previous Y to the next Y
            for nj in N_j:
                G16.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
            
    return G16

$Y_{t+1}[j]=\sqrt{0.000019+0.846 *\left(\bar{Y}_t\left[\mathcal{N}_j\right]^2+0.3 \bar{Y}_{t-1}\left[\mathcal{N}_j\right]^2+0.2 \bar{Y}_{t-2}\left[\mathcal{N}_j\right]^2+0.1 \bar{Y}_{t-3}\left[\mathcal{N}_j\right]^2\right)} W_{t+1}[j]$

In [158]:
def time_series_update_17(Y, t, j, N_j, W):
    Y_bar_t = mean_over_indices(Y[t], N_j)
    Y_bar_t_minus_1 = mean_over_indices(Y[t-1], N_j)
    Y_bar_t_minus_2 = mean_over_indices(Y[t-2], N_j)
    Y_bar_t_minus_3 = mean_over_indices(Y[t-3], N_j)

    squared_sum = (
        Y_bar_t**2 + 
        0.3 * Y_bar_t_minus_1**2 + 
        0.2 * Y_bar_t_minus_2**2 + 
        0.1 * Y_bar_t_minus_3**2
    )

    coefficient = math.sqrt(0.000019 + 0.846 * squared_sum)

    return coefficient * W[t][j]


In [159]:
def build_DAG_time_series_17(T, N_j, N):
    # Create a Directed Acyclic Graph (DAG) using networkx for the thirteenth formula
    G17 = nx.DiGraph()

    for t in range(T+1):
        for j in range(N):
            G17.add_node(f"Y[{t}][{j}]")
    
    
    # Add nodes and edges to the DAG based on the dependencies for the thirteenth formula
    for t in range(T):  # Start from t=1 since the formula depends on t and t-1
        for j in range(N):

            # Add edges from the current and previous three Y to the next Y
            for nj in N_j:
                G17.add_edge(f"Y[{t}][{nj}]", f"Y[{t+1}][{j}]")
                if t >= 1:
                    G17.add_edge(f"Y[{t-1}][{nj}]", f"Y[{t+1}][{j}]")
                if t >= 2:
                    G17.add_edge(f"Y[{t-2}][{nj}]", f"Y[{t+1}][{j}]")
                if t >= 3:
                    G17.add_edge(f"Y[{t-3}][{nj}]", f"Y[{t+1}][{j}]")
            
                
    return G17

In [160]:
# Simple simulation

# Initialize parameters
T = 200  # Time steps
N = 5   # number of j indices

In [161]:

Y1 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 1
Y2 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 2
Y3 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 3
Y4 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 4
Y5 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 5
Y6 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 6
Y7 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 7
Y8 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 8
Y9 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 9
Y10 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 10
Y11 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 11
Y12 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 12
Y13 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 13
Y14 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 14
Y15 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 15

x = [1] + [0 for _ in range(T)]
Y16 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 16
Y17 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 17






In [162]:
utils = {}
utils[1] = (build_DAG_time_series_1,time_series_update_1,range(T))
utils[2] = (build_DAG_time_series_2,time_series_update_2,range(9, T))
utils[3] = (build_DAG_time_series_3,time_series_update_3,range(2, T))
utils[4] = (build_DAG_time_series_4,time_series_update_4,range(1, T))
utils[5] = (build_DAG_time_series_5,time_series_update_5,range(T))
utils[6] = (build_DAG_time_series_6,time_series_update_6,range(2, T))
utils[7] = (build_DAG_time_series_7,time_series_update_7,range(9, T))
utils[8] = (build_DAG_time_series_8,time_series_update_8,range(2, T))
utils[9] = (build_DAG_time_series_9,time_series_update_9,range(1, T))
utils[10] = (build_DAG_time_series_10,time_series_update_10,range(T))
utils[11] = (build_DAG_time_series_11,time_series_update_11,range(T))
utils[12] = (build_DAG_time_series_12,time_series_update_12,range(1, T))
utils[13] = (build_DAG_time_series_13,time_series_update_13,range(1, T))
utils[14] = (build_DAG_time_series_14,time_series_update_14,range(T))
utils[15] = (build_DAG_time_series_15,time_series_update_15,range(T))
utils[16] = (build_DAG_time_series_16,time_series_update_16,range(T))
utils[17] = (build_DAG_time_series_17,time_series_update_17,range(3, T))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [117]:
#picle it
import pickle
with open('../data/known_ts_dict.pkl', 'wb') as handle:
    pickle.dump(to_pickle, handle)

In [1]:
lag = 3
rename_dict = {f"Y[{t}][{j}]": f"{j}_t-{lag-t}" for t in range(lag+1) for j in range(N)}

num_iterations = 250
observations_list = []

old_dags = []
dags = []

causal_dfs = []

for _ in range(num_iterations):
    amount_N_j = random.randint(1, 3)
    N_j = random.sample(range(N), amount_N_j)

    Y1 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 1

    dag = build_DAG_time_series_1(T=lag,N_j=N_j, N=N)
    dag = nx.relabel_nodes(dag, rename_dict)
    causal_df = get_causal_dfs(dag, N, lag)

    # Time series iteration for the first formula
    for t in range(T):
        for j in range(N):
            if t > 0:
                Y1[t+1][j] = time_series_update_1(Y1, t, j, N_j, W)
    observations = pd.DataFrame(Y1)
    
    observations_list.append(observations)
    old_dags.append(None)
    dags.append(dag)
    causal_dfs.append(causal_df)

    # Y2 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 1

    # dag = build_DAG_time_series_2(T=lag,N_j=N_j, N=N)
    # dag = nx.relabel_nodes(dag, rename_dict)
    # causal_df = get_causal_dfs(dag, N, lag)

    # # Time series iteration for the second formula
    # for t in range(9, T):  # Start from t=9 as the formula requires t-9
    #     for j in range(N):
    #         Y2[t+1][j] = time_series_update_2(Y2, t, j, N_j, W)
    # observations = pd.DataFrame(Y2)

    # observations_list.append(observations)
    # old_dags.append(None)
    # dags.append(dag)
    # causal_dfs.append(causal_df)

    Y3 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 1

    dag = build_DAG_time_series_3(T=lag,N_j=N_j, N=N)
    dag = nx.relabel_nodes(dag, rename_dict)
    causal_df = get_causal_dfs(dag, N, lag)

    # Time series iteration for the third formula
    for t in range(2, T):  # Start from t=2 as the formula requires t-2
        for j in range(N):
            Y3[t+1][j] = time_series_update_3(Y3, t, j, N_j, W)
    observations = pd.DataFrame(Y3)

    observations_list.append(observations)
    old_dags.append(None)
    dags.append(dag)
    causal_dfs.append(causal_df)

    Y4 = [[random.uniform(-1, 1) for _ in range(N)] for _ in range(T+1)]  # For time series 1

    dag = build_DAG_time_series_4(T=lag,N_j=N_j, N=N)
    dag = nx.relabel_nodes(dag, rename_dict)
    causal_df = get_causal_dfs(dag, N, lag)

    # Time series iteration for the fourth formula
    for t in range(1, T):  # Start from t=1 as the formula requires t-1
        for j in range(N):
            Y4[t+1][j] = time_series_update_4(Y4, t, j, N_j, W)
    observations = pd.DataFrame(Y4)
    
    observations_list.append(observations)
    old_dags.append(None)
    dags.append(dag)
    causal_dfs.append(causal_df)






NameError: name 'N' is not defined

In [41]:
import pickle
with open('../data/known_train.pkl', 'wb') as f:
    pickle.dump((observations_list,old_dags,dags,causal_dfs), f)

In [43]:
import pickle
with open('../data/known_test.pkl', 'wb') as f:
    pickle.dump((observations_list,old_dags,dags,causal_dfs), f)

In [22]:

# Time series iteration for the first formula
for t in range(T):
    for j in range(N):
        if t > 0:
            Y1[t+1][j] = time_series_update_1(Y1, t, j, N_j, W)

# Time series iteration for the second formula
for t in range(9, T):  # Start from t=9 as the formula requires t-9
    for j in range(N):
        Y2[t+1][j] = time_series_update_2(Y2, t, j, N_j, W)

# Time series iteration for the third formula
for t in range(2, T):  # Start from t=2 as the formula requires t-2
    for j in range(N):
        Y3[t+1][j] = time_series_update_3(Y3, t, j, N_j, W)

# Time series iteration for the fourth formula
for t in range(1, T):  # Start from t=1 as the formula requires t-1
    for j in range(N):
        Y4[t+1][j] = time_series_update_4(Y4, t, j, N_j, W)

for t in range(T):
    for j in range(N):
        Y5[t+1][j] = time_series_update_5(Y5, t, j, N_j, W)

for t in range(2, T):  # Start from t=2 as the formula requires t-2
    for j in range(N):
        Y6[t+1][j] = time_series_update_6(Y6, t, j, N_j, W)

for t in range(9, T):  # Start from t=9 as the formula requires t-9
    for j in range(N):
        Y7[t+1][j] = time_series_update_7(Y7, t, j, N_j, W)

for t in range(2, T):  # Start from t=2 as the formula requires t-2
    for j in range(N):
        Y8[t+1][j] = time_series_update_8(Y8, t, j, N_j, W)

for t in range(1, T):  # Start from t=1 as the formula requires t-1
    for j in range(N):
        Y9[t+1][j] = time_series_update_9(Y9, t, j, N_j, W)

for t in range(T):
    for j in range(N):
        Y10[t+1][j] = time_series_update_10(Y10, t, j, N_j, W)

for t in range(T):
    for j in range(N):
        Y11[t+1][j] = time_series_update_11(Y11, t, j, N_j, W)

for t in range(1, T):  # Start from t=1 as the formula requires t-1
    for j in range(N):
        Y12[t+1][j] = time_series_update_12(Y12, t, j, N_j, W)

for t in range(1, T):  # Start from t=1 as the formula requires t-1
    for j in range(N):
        Y13[t+1][j] = time_series_update_13(Y13, t, j, N_j, W)

for t in range(T):
    for j in range(N):
        Y14[t+1][j] = time_series_update_14(Y14, t, j, N_j, W)

for t in range(T):
    for j in range(N):
        Y15[t+1][j] = time_series_update_15(Y15, t, j, N_j, W)

for t in range(T):
    for j in range(N):
        Y16[t+1][j] = time_series_update_16(Y16, t, j, N_j, W, x[t])
    x[t+1] = update_x(x[t])

for t in range(3, T):  # Start from t=3 as the formula requires t-3
    for j in range(N):
        Y17[t+1][j] = time_series_update_17(Y17, t, j, N_j, W)




In [23]:

import pandas as pd
for series in [Y1, Y2, Y3, Y4, Y5, Y6, Y7, Y8, Y9, Y10, Y11, Y12, Y13, Y14, Y15]:
    print("Time Series:")
    print(pd.DataFrame(series))


Time Series:
            0         1         2         3         4
0   -0.829666 -0.367804 -0.213773 -0.676894  0.088800
1    0.276809  0.837437  0.459088  0.125813  0.560632
2    0.080687  0.038646  0.095609  0.049091 -0.038078
3    0.642569  0.592658  0.661731  0.696763  0.544996
4    1.033971  1.062560  1.135325  1.024772  1.109240
..        ...       ...       ...       ...       ...
196  0.631594  0.643213  0.656237  0.607578  0.636278
197  0.088202  0.164245  0.070851  0.165240  0.096407
198  0.588318  0.696368  0.605019  0.662888  0.549511
199  1.134469  1.081052  0.944110  1.025036  1.075492
200  1.504147  1.423861  1.365420  1.497940  1.381307

[201 rows x 5 columns]
Time Series:
            0         1         2         3         4
0   -0.022191  0.857372  0.420501  0.669459  0.436790
1    0.162919 -0.732696  0.331800  0.961134  0.910135
2    0.796398 -0.504056  0.128724  0.457778 -0.978301
3    0.422833  0.154235 -0.065627  0.803581  0.646240
4   -0.000419  0.336954 -0.84402